In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
today

datetime.date(2022, 1, 6)

In [2]:
format_dict = {
               'qty':'{:,}','SHARES':'{:,}','available_qty':'{:,}','volbuy':'{:,}','volsell':'{:,}','volbal':'{:,}',
               'price':'{:,.2f}','PRICE':'{:,.2f}','max_price':'{:,.2f}','min_price':'{:,.2f}',
               'buy_target':'{:,.2f}','sell_target':'{:,.2f}','fee':'{:,.2f}','vat':'{:,.2f}',
               'net':'{:,.2f}','profit':'{:,.2f}','cost':'{:,.2f}','volume':'{:,.2f}','beta':'{:,.2f}',
               'Q4':'{:,.3f}','Q3':'{:,.3f}','Q2':'{:,.3f}','Q1':'{:,.3f}','DIVIDEND':'{:,.3f}',
               'percent':'{:,.2f}%','PERCENT':'{:,.2f}%','yearly':'{:,.2f}%','pct':'{:,.2f}%',
              }

### Tables in the process

In [11]:
# Sells table in MySql portfolio database
sql = """
SELECT T.name, B.date AS buy_date, S.* FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

,name,buy_date,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
0,EPG,2021-09-23,897,1090,2022-01-06,12.30,50.92,3.56,"24,545.52",105,"2,095.90",9.34%,32.45%,103,None,0


In [15]:
# Buys table in MySql portfolio database
names = sells['name']
name = names.to_string(index=False)

sql = """
SELECT T.name, B.* FROM buys B
JOIN stocks T ON B.stock_id = T.id
ORDER BY B.id DESC
LIMIT 1"""
print(sql)

buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT T.name, B.* FROM buys B
JOIN stocks T ON B.stock_id = T.id
ORDER BY B.id DESC
LIMIT 1


,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,EPG,1090,62,2021-09-23,"2,000",11.20,46.37,3.25,Sold,"22,449.62",DTD,None


In [16]:
# Stocks table in MySql portfolio database
names = buys['name']
name = names.to_string(index=False)

sql = """
SELECT T.* FROM stocks T
WHERE name = '%s'"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conpf)
stocks.style.format(format_dict)


SELECT T.* FROM stocks T
WHERE name = 'EPG'


,id,name,category_id,website
0,62,EPG,3,www.epg.co.th


In [51]:
# Buy table in MySql stock database
names = sells['name']
name = names.to_string(index=False)

sql = """
SELECT * FROM buy
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.style.format(format_dict)


SELECT * FROM buy
WHERE name = 'DCC'



,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade


In [ ]:
# Dividend table in MySql stock database
names = sells['name']
name = names.to_string(index=False)

sql = """
SELECT * FROM dividend
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.style.format(format_dict)

In [ ]:
date = today.strftime('%Y-%m-%d')
date

### Sell transactions

### Begin of sell process to buy table in mysql stock database

In [27]:
sql = '''
SELECT T.name, S.* , B.net AS cost, B.qty 
FROM sells S
JOIN buys B
ON buy_id = B.id
JOIN stocks T
ON B.stock_id = T.id
WHERE S.date = '%s'
AND T.name = "DCC"
ORDER BY S.id DESC LIMIT 1
'''
sql = sql % today
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

,name,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id,cost,qty
0,DCC,896,1007,2022-01-06,3.00,248.40,17.39,"119,734.21",114,"3,477.28",2.99%,9.58%,104,None,0,"116,256.93","40,000"


In [28]:
names = sells['name']
name = names.to_string(index=False)
name

'DCC'

In [29]:
sr_qty = sells['qty']
qty_str = sr_qty.to_string(index=False)
qty = int(qty_str) * -1
qty

-40000

In [30]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy


SELECT * 
FROM buy 
WHERE name = 'DCC'



,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,DCC,2021-09-14,40000.0,2.9,0.0,0.0,1,0.184,4,A1


In [31]:
sqlUpd = '''
UPDATE buy
SET volbuy = volbuy + %s
WHERE name = "%s"'''
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)

rp = const.execute(sqlUpd)
rp.rowcount


UPDATE buy
SET volbuy = volbuy + -40000
WHERE name = "DCC"


1

In [32]:
sqlDel = '''
DELETE FROM buy
WHERE volbuy = 0'''
rp = const.execute(sqlDel)
rp.rowcount

1

### End of sell process to buy table in mysql stock database

### Begin of sell process to stocks table in sqlite port_lite database

In [40]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'DCC'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,519,DCC,3.40,2.22,I,0.00,2.92,91.20,0.57,2.90,"40,000",-7,8,"40,000",0,12,CP1S,SET


sqlUpd = '''
UPDATE stocks
SET available_qty = available_qty + %s
WHERE name = "%s"'''
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)

rp = conlite.execute(sqlUpd)
rp.rowcount

### End of sell process to stocks table in sqlite port_lite database

### Begin of Update table dividend in mysql stock database for sells transactions

In [34]:
sql = """
SELECT * 
FROM dividend 
WHERE name = "%s" 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = "DCC" 



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,DCC,0.044,0.040,0.050,0.050,0.184,0.00,0.00%,"40,000.0",2022-04-05,2021-12-03,,0


In [35]:
sqlUpd = '''
UPDATE dividend
SET shares = shares + %s
WHERE name = "%s"'''
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)

rp = const.execute(sqlUpd)
rp.rowcount


UPDATE dividend
SET shares = shares + -40000
WHERE name = "DCC"


1

In [38]:
sqlDel = """
DELETE FROM dividend
WHERE name = '%s' AND shares = 0
"""
sqlDel = sqlDel % name
print(sqlDel)

rp = const.execute(sqlDel)
rp.rowcount


SELECT * 
FROM dividend 
WHERE name = "DCC" 



1

### End of Update table dividend in mysql stock database for sells transactions

### Buys transactions

### Begin of buy process to buy table in mysql stock database

In [44]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy


SELECT * 
FROM buy 
WHERE name = 'DCC'



,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade


In [47]:
new_cost = 0  # new average cost must be calculated manually beforehand

sqlUpd = '''
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = "%s"'''
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + -40000, price = 0
WHERE name = "DCC"


In [10]:
rp = const.execute(sqlUpd)
rp.rowcount

1

### End of buy process to buy table in mysql stock database

### Begin of buy process to stocks table in sqlite port_lite database

In [48]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'DCC'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,519,DCC,3.40,2.22,I,0.00,2.92,91.20,0.57,2.90,"40,000",-7,8,0,0,12,CP1S,SET


In [49]:
sqlUpd = '''
UPDATE stocks
SET available_qty = available_qty + %s
WHERE name = "%s"'''
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)

rp = conlite.execute(sqlUpd)
rp.rowcount


UPDATE stocks
SET available_qty = available_qty + -40000
WHERE name = "DCC"


1

### End of buy process to stocks table in sqlite port_lite database

### Begin of Update table dividend in mysql stock database for buy transactions

In [50]:
sql = """
SELECT * 
FROM dividend 
WHERE name = "%s" 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = "DCC" 



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL


In [ ]:
sqlUpd = '''
UPDATE dividend
SET shares = shares + %s
WHERE name = "%s"'''
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)

rp = const.execute(sqlUpd)
rp.rowcount

### End of Update table dividend in mysql stock database for buy transactions